In [ ]:
import cv2
import numpy as np
import gradio as gr
import traceback

# Helpers: dtype + normalization


def _to_uint8(img):
    if img is None:
        return None
    if np.issubdtype(img.dtype, np.floating):
        if img.max() <= 1.5:
            img = img * 255.0
    return np.clip(img, 0, 255).astype(np.uint8)

def to_gray_uint8(img):
    if img is None:
        return None
    img = _to_uint8(img)
    if img.ndim == 3 and img.shape[2] == 4:
        img = img[:, :, :3]
    if img.ndim == 3:
        img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    return img.astype(np.uint8)

def to_float01(gray_u8):
    return gray_u8.astype(np.float32) / 255.0

def gray_u8_to_rgb_u8(gray_u8):
    return cv2.cvtColor(gray_u8, cv2.COLOR_GRAY2RGB)

def ensure_rgb_uint8(img):
    if img is None:
        return None
    img = _to_uint8(img)
    if img.ndim == 2:
        img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
    if img.ndim == 3 and img.shape[2] == 4:
        img = img[:, :, :3]
    return img

def ref_has_chroma(ref_bgr_u8, sat_thresh=5.0):
    hsv = cv2.cvtColor(ref_bgr_u8, cv2.COLOR_BGR2HSV)
    return hsv[:, :, 1].mean() >= sat_thresh


# Colorization methods


def create_gradient_lut():
    lut = np.zeros((256, 3), dtype=np.float32)
    for i in range(256):
        t = i / 255.0
        lut[i] = [t, 1 - abs(2*t - 1), 1 - t]
    return (lut * 255).astype(np.uint8)

LUT_GRADIENT = create_gradient_lut()

def apply_lut(gray_u8, lut):
    return lut[gray_u8]

def band_colorization(gray_u8):
    h, w = gray_u8.shape
    out = np.zeros((h, w, 3), dtype=np.uint8)
    out[gray_u8 < 85] = [0, 0, 150]
    out[(gray_u8 >= 85) & (gray_u8 < 170)] = [255, 255, 0]
    out[gray_u8 >= 170] = [255, 0, 0]
    return out

def kmeans_colorization(gray_u8, k=4):
    """Cluster-based pseudo-color (uint8 in, uint8 RGB out)."""
    Z = gray_u8.reshape((-1, 1)).astype(np.float32)
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)

    _, labels, centers = cv2.kmeans(
        Z, k, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS
    )

    labels = labels.flatten()
    centers = centers.astype(np.uint8).flatten()

    # Same extended palette you had originally (safe for k up to 7)
    palette = np.array([
        [0, 0, 128],
        [0, 128, 255],
        [0, 255, 255],
        [0, 255, 0],
        [150, 0, 150],
        [200, 200, 0],
        [255, 128, 0]
    ], dtype=np.uint8)

    # Guard: if k > palette length, repeat palette (still deterministic)
    if k > len(palette):
        reps = int(np.ceil(k / len(palette)))
        palette = np.tile(palette, (reps, 1))

    colors = palette[:k]
    colored = colors[labels].reshape(gray_u8.shape[0], gray_u8.shape[1], 3)

    return colored, centers


def color_transfer_reinhard(ref_bgr, gray_u8):
    ref_lab = cv2.cvtColor(ref_bgr, cv2.COLOR_BGR2LAB)
    Lr, ar, br = ref_lab[:, :, 0], ref_lab[:, :, 1], ref_lab[:, :, 2]

    lut_a = np.interp(
        np.arange(256),
        np.unique(Lr),
        [ar[Lr == l].mean() for l in np.unique(Lr)],
        left=128, right=128
    )
    lut_b = np.interp(
        np.arange(256),
        np.unique(Lr),
        [br[Lr == l].mean() for l in np.unique(Lr)],
        left=128, right=128
    )

    lab = np.zeros((*gray_u8.shape, 3), dtype=np.uint8)
    lab[:, :, 0] = gray_u8
    lab[:, :, 1] = lut_a[gray_u8].astype(np.uint8)
    lab[:, :, 2] = lut_b[gray_u8].astype(np.uint8)

    return cv2.cvtColor(lab, cv2.COLOR_LAB2BGR)

def cars_colorization_and_blend(gray_u8, alpha=0.7):
    _, mask = cv2.threshold(gray_u8, 160, 255, cv2.THRESH_BINARY)
    num_labels, labels, stats, _ = cv2.connectedComponentsWithStats(mask)

    color_img = cv2.cvtColor(gray_u8, cv2.COLOR_GRAY2BGR)
    palette = [
        (20,20,220),(225,105,65),(0,215,255),
        (50,205,50),(0,140,255)
    ]

    idx = 0
    for i in range(1, num_labels):
        if 150 <= stats[i, cv2.CC_STAT_AREA] <= 800:
            color_img[labels == i] = palette[idx % len(palette)]
            idx += 1

    solid = cv2.cvtColor(color_img, cv2.COLOR_BGR2RGB)

    gray_bgr = cv2.cvtColor(gray_u8, cv2.COLOR_GRAY2BGR)
    blended = gray_bgr.copy()
    blended[labels > 0] = cv2.addWeighted(
        gray_bgr[labels > 0], 1 - alpha,
        color_img[labels > 0], alpha, 0
    )
    return solid, cv2.cvtColor(blended, cv2.COLOR_BGR2RGB)

def adjust_saturation(img, factor):
    factor = float(np.clip(factor, 0.0, 5.0))
    hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HSV).astype(np.float32)
    hsv[:, :, 1] = np.clip(hsv[:, :, 1] * factor, 0, 255)
    return cv2.cvtColor(hsv.astype(np.uint8), cv2.COLOR_HSV2RGB)


# Gradio interface logic


METHODS = {
    "pseudo-color lut": "pseudo-color lut",
    "band colorization": "band colorization",
    "k-means colorization": "k-means colorization",
    "reinhard color transfer": "reinhard color transfer",
    "solid colors": "solid colors",
    "blended with grayscale": "blended with grayscale",
}

def colorize_interface(gray_input, method, saturation, ref_image, alpha_blend):
    try:
        if gray_input is None:
            return None

        saturation = 1.0 if saturation is None else float(saturation)
        alpha_blend = 0.7 if alpha_blend is None else float(alpha_blend)

        gray_u8 = to_gray_uint8(gray_input)
        gray_rgb = gray_u8_to_rgb_u8(gray_u8)

        key = METHODS.get(method.lower(), method.lower())

        if key == "pseudo-color lut":
            out = apply_lut(gray_u8, LUT_GRADIENT)

        elif key == "band colorization":
            out = band_colorization(gray_u8)

        elif key == "k-means colorization":
            out, _ = kmeans_colorization(gray_u8)

        elif key == "reinhard color transfer":
            ref = ensure_rgb_uint8(ref_image)
            if ref is None:
                ref = cv2.applyColorMap(gray_u8, cv2.COLORMAP_JET)
            else:
                ref = cv2.cvtColor(ref, cv2.COLOR_RGB2BGR)
            out = cv2.cvtColor(color_transfer_reinhard(ref, gray_u8), cv2.COLOR_BGR2RGB)

        elif key == "solid colors":
            out, _ = cars_colorization_and_blend(gray_u8, alpha_blend)

        elif key == "blended with grayscale":
            _, out = cars_colorization_and_blend(gray_u8, alpha_blend)

        else:
            out = gray_rgb

        out = adjust_saturation(out, saturation)
        return out.astype(np.uint8)

    except Exception:
        raise gr.Error(traceback.format_exc())


# UI


with gr.Blocks() as demo:
    gr.Markdown("## B&W → Colorization Filters (All Methods from Notebook)")

    with gr.Row():
        with gr.Column():
            method = gr.Radio(
                choices=[
                    "Pseudo-Color LUT",
                    "Band Colorization",
                    "K-Means Colorization",
                    "Reinhard Color Transfer",
                    "Solid Colors",
                    "Blended with Grayscale",
                ],
                value="Pseudo-Color LUT",
                label="Choose filter / method"
            )
            saturation = gr.Slider(0, 2, value=1, step=0.05, label="Saturation factor")
            alpha = gr.Slider(0, 1, value=0.7, step=0.05, label="Region blending alpha")
            ref = gr.Image(label="Reference image (Reinhard)", type="numpy")
            btn = gr.Button("Apply Filter")

        with gr.Column():
            inp = gr.Image(label="Input B&W image", type="numpy")
            out = gr.Image(label="Colorized output", type="numpy")

    btn.click(colorize_interface, [inp, method, saturation, ref, alpha], out)

if __name__ == "__main__":
    demo.launch(debug=True)


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://ffa751fd771b1a4083.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
